# OdC's 2025 Temperature Geovisor

This notebook loads each city's temperature data, processes and __creates the .htmls needed for kepler visualization.__

* __Important note: Worked fine with keplergl v0.3.2, but not with kepler v.0.3.7.__
* __Final parameters used for Temperature Visor:__
  * __res__: 10
  * __filter_urban__: True
  * __palette__: palette_2025
  * __stroke__: 0.05, white
  * __opacity__: 0.10

In [1]:
# Temperature anomaly 2025 OdC's social media publications (Satellite background)
# Hotter 3: "#ca0020" (Red)
# Hotter 2: "#e66e61" (Light red)
# Hotter 1:"#f5c0a9" (Very light red)
# Middle: "#f7f7f7" (Nearly white)
# Colder 1: "#b4d6e6" (Very light blue)
# Colder 2: "#63a9cf" (Light blue)
# Colder 3: "#0571b0" (Blue)

## Import libraries

In [2]:
from pathlib import Path

current_path = Path().resolve()

for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break

print(project_root)

/home/jovyan/accesibilidad-urbana


In [3]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt

# Imports que venían en el Notebook 15-min-city > 13-15-min-kepler-test.ipynb
import io
#import boto3
from keplergl import KeplerGl

# Classify data using Natural Breaks
import mapclassify

# Correlation calc
import scipy.stats as stats

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(project_root))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Config notebook

In [4]:
test = True
if test:
    city_lst = ['Monterrey'] # Gets overwritten if test = False
    res = 10
else:
    res = 10

projected_crs = 'EPSG:6372'
# Processing - Filter for urban areas only?
filter_urban = True
# Processing - If running CDMX (capital city), add ZMVM (metropolitan area)
merge_capital = True

# Directory where .html files are saved
map_output_dir = str(project_root) + f"/data/processed/visor_temperature/"

## Find all available cities (If test = False, rewrites city_list)

In [5]:
if test == False:
    # Find all available cities
    temp_schema = 'raster_analysis'
    temp_table = 'temperature_analysis_hex'
    scan_res = 8
    query = f"SELECT city FROM {temp_schema}.{temp_table} WHERE \"res\" = '{scan_res}\'"
    temperature_df = aup.df_from_query(query)
    city_lst = list(temperature_df.city.unique())
    del temperature_df

# If merge_capital, drop ZMVM in order to load ZMVM when loading CDMX during processing.
if merge_capital and ('CDMX' in city_lst) and ('ZMVM' in city_lst):
    print("--"*30)
    print("merge_capital IS SET TO TRUE: City list contains both CDMX and ZMVM. Will merge both databases and create a unified temperature anomaly.")
    print("--"*30)
    city_lst.remove('ZMVM')

print(f"{len(city_lst)} cities to run at res {res}.")
city_lst

1 cities to run at res 10.


['Monterrey']

## Create Kepler HTMLs files by city

In [6]:
i=0
for city in city_lst:

    try:
        print("--"*30)
        print(f"--- STARTING CITY {i}/{len(city_lst)}: {city}.")
    
        # 1.0 --- --- --- LOAD DATA --- --- ---
        
        # 1.1 --- LOAD URBAN HEXS
        print(f"Loading {city}'s urban hexs.")
        # Load data
        hex_schema = 'hexgrid'
        hex_table = f'hexgrid_{res}_city_2020'
        # Load city's hexs filtering for urban areas if required
        if filter_urban:
            hex_type = 'urban'
            query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = '{city}\' AND \"type\" = '{hex_type}\'"
        else:
            query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = '{city}\'"
        hex_gdf = aup.gdf_from_query(query, geometry_col='geometry')
        
        # SPECIFIC CASE - Merge capital's hexs (CDMX + ZMVM)
        if merge_capital and (city == 'CDMX'):
            print("MERGING CDMX + ZMVM hexs.")
            # Load ZMVM's hexs filtering for urban areas if required
            if filter_urban:
                hex_type = 'urban'
                query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = 'ZMVM\' AND \"type\" = '{hex_type}\'"
            else:
                query = f"SELECT hex_id_{res}, geometry FROM {hex_schema}.{hex_table} WHERE \"city\" = 'ZMVM\'"
            metro_hex_gdf = aup.gdf_from_query(query, geometry_col='geometry')
            # Merge CDMX and ZMVM hexs
            hex_gdf = pd.concat([hex_gdf,metro_hex_gdf])
            
        # Read and format cols
        hex_gdf['res'] = res
        hex_gdf.rename(columns={f'hex_id_{res}':'hex_id'},inplace=True)
        hex_gdf.to_crs(projected_crs,inplace=True)
        # List all unique hex_ids
        hexid_lst = list(hex_gdf.hex_id.unique())
        del hex_gdf
        
        # 1.2 --- LOAD TEMPERATURE DATA
        print(f"Loading {city}'s Temperature data.")
        temp_schema = 'raster_analysis'
        temp_table = 'temperature_analysis_hex'
        query = f"SELECT * FROM {temp_schema}.{temp_table} WHERE \"city\" = '{city}\' AND \"res\" = '{res}\'"
        temperature_gdf = aup.gdf_from_query(query, geometry_col='geometry')

        # SPECIFIC CASE - Merge capital's temperature hexs (CDMX + ZMVM)
        if merge_capital and (city == 'CDMX'):
            print("MERGING CDMX + ZMVM TEMPERATURE.")
            # Load metro area's temperature hexs
            query = f"SELECT * FROM {temp_schema}.{temp_table} WHERE \"city\" = 'ZMVM\' AND \"res\" = '{res}\'"
            metro_temp_gdf = aup.gdf_from_query(query, geometry_col='geometry')
            # Merge capital and metro area temperature hexs
            temperature_gdf = pd.concat([temperature_gdf,metro_temp_gdf])
            # Drop duplicated temperature hexs (Both in CDMX and ZMVM)
            temperature_gdf.drop_duplicates(subset="hex_id",
                                            inplace=True)
    
        temperature_gdf.to_crs(projected_crs,inplace=True)
        # Filter for urban areas if required
        if filter_urban:
            temperature_gdf = temperature_gdf.loc[temperature_gdf.hex_id.isin(hexid_lst)].copy()
        # Inf values check
        infs = temperature_gdf.loc[np.isinf(temperature_gdf['temperature_mean'])]
        if len(infs)>0:
            print(f"WARNING: Dropping {len(infs)} hexs res {res} because of inf values.")
            temperature_gdf = temperature_gdf.loc[~np.isinf(temperature_gdf['temperature_mean'])].copy()
        
        # 2.0 --- --- --- CALCULATIONS AND DATA TREATMENT --- --- ---
        
        # 2.1 --- CALCULATE TEMPERATURE ANOMALY
        print("DATA TREATMENT - Calculating temperature anomaly.")
        # Calculate anomaly by hex (differential between mean in each hex and city mean)
        mean_city_temperature = temperature_gdf.temperature_mean.mean()
        temperature_gdf['temperature_anomaly'] = temperature_gdf['temperature_mean'] - mean_city_temperature
        
        # 2.2 --- CATEGORIZE TEMPERATURE ANOMALY USING JENKS
        print("DATA TREATMENT - Categorize temperature anomaly using jenks.")
        # Calculate Natural Breaks (Jenks) cuts (using 7 classes)
        classifier = mapclassify.NaturalBreaks(y=temperature_gdf['temperature_anomaly'], k=7)
        # Add class to each hex
        temperature_gdf['anomaly_class'] = classifier.yb
        # Rename classes to be between -3 (colder) and 3 (hotter)
        temperature_gdf['anomaly_class'] -= 3
        
        # 2.3 --- SET CATEGORY'S NAME TO BE DISPLAYED ON GEOVISOR (Creates column)
        print("DATA TREATMENT - Renaming categories.")
        # Extract array of Natural Breaks upper bounds
        bins_array = classifier.bins
        # Set number of decimals for temperature bins to be show
        decimals = 2
        # Rename anomaly classes to categories containing temperature bounds for current city
        classes_dict = {-3:f"7. Lo más fresco (Menos de {round(bins_array[0],decimals)}°)",
                                -2:f"6. Más fresco ({round(bins_array[1],decimals)}° a {round(bins_array[0],decimals)}°)",
                                -1:f"5. Fresco ({round(bins_array[2],decimals)}° a {round(bins_array[1],decimals)}°)",
                                0:f"4. Cercano al promedio de la ciudad ({round(bins_array[2],decimals)}° a {round(bins_array[3],decimals)}°)",
                                1:f"3. Caliente ({round(bins_array[3],decimals)}° a {round(bins_array[4],decimals)}°)",
                                2:f"2. Más caliente ({round(bins_array[4],decimals)}° a {round(bins_array[5],decimals)}°)",
                                3:f"1. Lo más caliente (Más de {round(bins_array[5],decimals)}°)"
                               }
        temperature_gdf['anomaly_label'] = temperature_gdf['anomaly_class'].map(classes_dict)
        
        # 2.4 --- CONVERT TO CATEGORICAL ORDER
        print("DATA TREATMENT - Setting categories as categorical values.")
        # Define order and convert col into ordered category
        temperature_categories = list(classes_dict.values())
        temperature_gdf['anomaly_label'] = pd.Categorical(temperature_gdf['anomaly_label'], categories=temperature_categories, ordered=True)
        # Force categorical order
        temperature_gdf.sort_values(by='anomaly_label', inplace=True)

        # 2.5 --- CONSIDER TEMPERATURE ANOMALY DIRECTLY, ROUNDING VALUE
        temperature_gdf['temperature_anomaly_rounded'] = temperature_gdf['temperature_anomaly'].round(2)
        
        # 3.0 --- --- --- PREPARE GDF FOR KEPLER CONFIGURATION --- --- ---
        print("GDF PREPARATION - Preparing hex_kepler.")
        # Rename columns of interest
        main_col = f'Anomalía de temperatura con respecto al promedio en {city}.' #Column with temperature anomaly classified
        hovering_col = "Anomalía de temperatura en el hexágono." # Column with specific hex data
        rename_columns = {'anomaly_label':main_col,
                          'temperature_anomaly_rounded':hovering_col}
        hex_kepler = temperature_gdf.copy()
        hex_kepler.rename(columns=rename_columns,inplace=True)
        # Keep columns of interest and geometry
        hex_kepler = hex_kepler[[main_col,hovering_col,'geometry']]

        # PREPARE HEX KEPLER - Kepler not loading if it is projected_crs
        if hex_kepler.crs != "EPSG:4326":
            hex_kepler.to_crs("EPSG:4326",inplace=True)
            print(f"GDF PREPARATION - Changed {city}'s hex_kepler crs to EPSG:4326.")
        
        # 4.0 --- --- --- KEPLER CONFIGURATION --- --- ---
        # CREATE CONFIGURATION
        print(f"KEPLER CONFIGURATION - Starting {city}'s map config.")
        config, config_idx = aup.kepler_config()
        
        # LAYER CONFIGURATION - Set layer to be visualized on map
        print("Layer configuration.")
        config["config"]["visState"]["layers"][0]["visualChannels"]["colorField"]["name"] = main_col
        
        # PALETTE CONFIGURATION - Set the color palette of the visualized layer
        print("Palette configuration.")
        # Temperature anomaly 2025 OdC's social media publications (Satellite background)
        sm2025_palette = ["#ca0020", #(Red)
                          "#e66e61", #(Light red)
                          "#f5c0a9", #(Very light red)
                          "#f7f7f7", #(Nearly white)
                          "#b4d6e6", #(Very light blue)
                          "#63a9cf", #(Light blue)
                          "#0571b0"] #(Blue)       
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["colorRange"] = {"name": "sm2025_palette",
                                                                                          "type": "custom",
                                                                                          "category": "Custom",
                                                                                          "colors": sm2025_palette,
                                                                                          "reversed": False}
        
        # STROKE CONFIGURATION - Set stroke width and color [Final decision: 0.05 white]
        print("Stroke configuration.")
        # Black --> "#000000" --> [0, 0, 0]
        # White --> "#ffffff" --> [255, 255, 255]
        stroke=0.05
        strokecolor = [255, 255, 255]
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["thickness"] = stroke #aup.kepler_config() value: 0.5
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["strokeColor"] = strokecolor #aup.kepler_config() value: [28, 27, 27]

        # OPACITY CONFIGURATION - Set object's opacity [Final decision: 0.10]
        print("Opacity configuration.")
        fillopacity = 0.10
        config["config"]["visState"]["layers"][0]["config"]["visConfig"]["opacity"] = fillopacity #aup.kepler_config() value: 0.85
        
        # FUNCTIONALITIES CONFIGURATION - Turning on/off map functionalities
        print("Functionalities configuration.")
        # Turn on geocoder (Search tab)
        config["config"]["visState"]["interactionConfig"]["geocoder"]["enabled"] = True
        # Stop user from activating manually dual map viewer
        config["config"]["splitMaps"] = []
        # Force turn on legend
        config["config"]["visState"]["legend"] = {"active": True}
        # FUNCTIONALITIES CONFIGURATIONS ALREADY TURNED OFF IN FUNCTION (Written here as reminder)
        # Turn off dual map viewer #aup.kepler_config() value:False
        #config["config"]["mapState"]["isSplit"] = False
        # Turn off brush #aup.kepler_config() value:False
        #config["config"]["visState"]["interactionConfig"]["brush"]["enabled"] = False
        # Turn off coordinate #aup.kepler_config() value:False
        #config["config"]["visState"]["interactionConfig"]["coordinate"]["enabled"] = False
        
        # STARTING POINT CONFIGURATION - Set map's starting point for each city
        print("Starting point configuration.")
        longitude = hex_kepler.dissolve().geometry.centroid.x
        latitude = hex_kepler.dissolve().geometry.centroid.y
        config["config"]["mapState"]["latitude"] = latitude[0]
        config["config"]["mapState"]["longitude"] = longitude[0]
        
        # HOVERING CONFIGURATION - Set data to show when hovering over hex
        print("Hovering configuration.")
        config["config"]["visState"]["interactionConfig"]["tooltip"]["fieldsToShow"] = {"Análisis de hexágono": [hovering_col]}  
        
        # END OF CONFIGURATION - Add configuration and data to html file
        print("Add configuration.")
        map_city = KeplerGl(height=800)
        map_city.config = config
        map_city.add_data(hex_kepler, name='Análisis de hexágono')
        
        print(f"KEPLER CONFIGURATION - Finished {city}'s map config.")
        
        # 4.0 --- --- --- SAVE HTML --- --- ---
        if test == True:
            file_name = f'TestTemperature_{city}_{res}_op{fillopacity}_strk{stroke}_sm2025_palette.html'
            map_city.save_to_html(file_name = map_output_dir+f"tests/{file_name}", read_only=False)
            print(f"SAVING (TEST) - Saved {city}'s map config html.")
        else:
            # SPECIFIC CASE - Merge capital's Temperature (CDMX + ZMVM) file name
            if merge_capital and (city == 'CDMX'):
                file_name = f'VisorTemperature_{city}-ZMVM_res{res}.html'
            else:
                file_name = f'VisorTemperature_{city}_res{res}.html'
            map_city.save_to_html(file_name = map_output_dir+f"{file_name}", read_only=False)
            print(f"SAVING (FINAL) - Saved {city}'s map config html file.")
    
    # In case of error while running city
    except:
        print("--"*30)
        print(f"ERROR: {city}.")
        print("--"*30)

    i+=1

------------------------------------------------------------
--- STARTING CITY 0/1: Monterrey.
Loading Monterrey's urban hexs.
Loading Monterrey's Temperature data.
DATA TREATMENT - Calculating temperature anomaly.
DATA TREATMENT - Categorize temperature anomaly using jenks.
DATA TREATMENT - Renaming categories.
DATA TREATMENT - Setting categories as categorical values.
GDF PREPARATION - Preparing hex_kepler.
GDF PREPARATION - Changed Monterrey's hex_kepler crs to EPSG:4326.
KEPLER CONFIGURATION - Starting Monterrey's map config.
Layer configuration.
Palette configuration.
Stroke configuration.
Opacity configuration.
Functionalities configuration.
Starting point configuration.


/tmp/ipykernel_20961/1902507556.py:192: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longitude = hex_kepler.dissolve().geometry.centroid.x
/tmp/ipykernel_20961/1902507556.py:193: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitude = hex_kepler.dissolve().geometry.centroid.y


Hovering configuration.
Add configuration.
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
KEPLER CONFIGURATION - Finished Monterrey's map config.
Map saved to /home/jovyan/accesibilidad-urbana/data/processed/visor_temperature/tests/TestTemperature_Monterrey_10_op0.1_strk0.05_sm2025_palette.html!
SAVING (TEST) - Saved Monterrey's map config html.
